# Python生成器
python中的生成器本质上是一种特殊的迭代器，关于迭代器可以自行搜索（主要是实现了\__iter__和\__next__方法）。
### Python中的惰性求值
想弄清楚生成器必须要理解python中的惰性求值特性。惰性求值是指，在程序运行中并不会一次计算所有结果，而是根据需要计算部分结果。很多语言都有惰性求值的特性，在python比如：

In [4]:
result = None and "abcd"
result1 = None or "abcd"
print(result, result1)
tmp = iter(range(10))
print(next(tmp), next(tmp))

None abcd
0 1


惰性求值在逻辑判断中尤为突出。上述关于tmp的代码叶可以改写为：  
```for i in range(10)
    print(i)
```  
关键字for代替我们做了iter和next的工作，并且最后会自动捕捉StopIteration异常。  
迭代器在使用结果的时候才会进行一次计算获得一个返回值，这一特性会节省内存等资源的开销。

### 生成器函数和生成器
生成器函数是一类特殊的函数，它使用yeild关键字，并且返回一个生成器。以经典的斐波那契数列为例：

In [16]:
def test(num):
    print("func test")
    a = 0
    b = 1
    label = 0
    while label < num:
        yield a+b
        a, b = b, a+b
        label += 1
gen = test(5)
print("*"*10)
for t in gen:
    print(t)

**********
func test
1
2
3
5
8


这里在执行gen=test(5)时，函数并没有执行（没有输出）。而是在执行for循环的时候才有输出，这是因为赋值语句仅仅是返回了一个生成器而没有启动它，必须使用next（generator）才能启动（后续会讲到使用send方法也可以启动一个生成器），让函数体执行到yield语句并返回关键字yield之后的内容，然后等待下一次的next调用（每次调用next方法会将函数停止在yeild语句，会保存函数的执行状态（即参数的值），下一次会从yeild语句继续执行，直到碰到yield语句或生成器执行完毕引发StopIteration异常）。这里使用for循环的目的就是自动的调用next方法和捕捉StopIteration异常。在python3之前，生成器并不支持return关键字，在py3之后return关键字会引发StopIteration异常。

In [19]:
def test():
    print("func test")
    a = 0
    b = 1
    label = 0
    while 1:
        if label > 3:
            return 'End'
        yield a+b
        a, b = b, a+b
        label += 1

gen = test()
print(next(gen))
print(next(gen))
print(next(gen))
print(next(gen))
print(next(gen))

func test
1
2
3
5


StopIteration: End

可以看到迭代4次之后生成器终止，这里要注意的是return语句会直接引发StopIteration异常，但是return的值并不会被返回，上例里”End“并没有被输出。而是把return返回值当做异常的信息。如果我们想获得这个返回值怎么做？答案是可以使用异常处理。

In [30]:
def test():
    print("func test")
    a = 0
    b = 1
    label = 0
    while 1:
        if label > 1:
            return 'End'
        yield a+b
        a, b = b, a+b
        label += 1
gen = test()
try:
    print(next(gen))
    print(next(gen))
    print(next(gen))
except StopIteration as err:
    print(err)

func test
1
2
End


生成器可以无限迭代下去，但是只能迭代一次。如果想再次使用，可以生成一个新的生成器，或者使用send方法来传递参数给生成器来重置生成器。

In [51]:
def test():
    print("func test")
    a = 0
    b = 1
    label = 0
    while 1:
        tmp = yield a+b
        label = tmp or label
        if label > 1:
            return 'End'
        a, b = b, a+b
        label += 1
gen = test()
for i in gen:
    print(i)
#next(gen) #这里会引发StopIteration异常，如果想再次使用可以再次获取一个生成器。
gen = test()
for i in gen:
    print(i)
#或者我们可以使用send方法来重置生成器。由于for循环会直接将生成器执行结束，所以需要新生成一个生成器来进行测试。
gen = test()
print(next(gen))
print(next(gen))
#print(next(gen)) #这里不能再继续执行next否则会引发迭代结束异常。
print(gen.send(True))
print(gen.send(True))
print(gen.send(True))
print(gen.send(2))



func test
1
2
3
func test
1
2
3
func test
1
2
3
5
8


StopIteration: End

gen.send方法会将值传递进生成器，并作为yeild语句的返回值赋值给tmp。next()方法相当于gen.send(None)，但是需要注意一点，如果使用send方法启动生成器(之前的例子都是使用next方法启动)，那么参数只能是None，后续的send方法才可以正常传递参数进入生成器。除了send方法，生成器还有throw和close方法，分别向生成器抛入一个异常和关闭生成器。close方法在生成器内部引发一个GeneratorEixt异常。

In [114]:
def Test():
    while 1:
        try:
            yield 1
        except ValueError:
            print("除数不能为0")
        except GeneratorExit:
            print("关闭生成器")
            break
            #raise StopIteration

gen = Test()
print(next(gen))
print(gen.throw(ValueError))
print(next(gen))
gen.close()
#next(gen)

1
除数不能为0
1
1
关闭生成器


当使用close方法时，生成器会引发GeneratorExit异常，即使没有显示的捕获此异常，也不会抛向主程序。如果主程序退出，此时生成器没有终止也会抛出GeneratorExit异常。 当生成器启动，又引发GeneratorExit异常之后，生成器继续执行直到自身结束，如果中间再次碰到yield语句则会引发RuntimeError。如果close之后再次使用next方法，则会引发StopIteration异常。Exception,RuntimeError和GeneratorExit继承自BaseException，而StopIteration继承自Exception。GeneratorExit异常主要是帮助程序员解决一些资源收尾工作。
### yield from关键字
当生成器中含有return语句时，我们想获得这个返回值只能间接使用异常处理来获取，除此之外还可以使用关键字yeild from。py3.3以后版本新增yield from关键字，本意是简化生成器的嵌套使用。yield from后需要跟一个迭代器对象。

In [125]:
def test():
    result = yield from fab()
    while 1:
        yield result

def fab():
    a,b = 0,1
    label = 0
    while 1:
        if label > 3:
            return "end fab"
        tmp = yield a+b
        if tmp:
            print("重置label")
            label = 0
        a, b = b, a+b
        label += 1
gen = test()
print(next(gen))
print(next(gen))
print(gen.send(1))
print(next(gen))
print(next(gen))
print(next(gen))
print(next(gen))

1
2
重置label
3
5
8
end fab
end fab


yield from相当于在调用者和子生成器建立了一个桥梁，此时中间生成器祖塞，调用者直接向子生成器send参数、获得返回值或者throw异常，而不需要经过中间生成器。yield from会自动捕获StopIteration异常，并且将异常的值当做yeild from表达式的值进行返回。